In [48]:
base = '/u/ebanner/Classes/nlp/Project/irony-classifier'

In [ ]:
base = '/u/npockrus/NLP/finalProject/venv/src/irony-classifier'

# Load Conservative Comments

In [49]:
cd /{base}/data/conservative/features/text+sentiment+subreddit+label/

/v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/data/conservative/features/text+sentiment+subreddit+label


In [50]:
import pickle
import scipy
import numpy as np

with open('features.p', 'r') as f:
    data = pickle.load(f)

subreddits = np.array([data[sentence]['subreddits'] for sentence in sorted(data)])
sentiments = np.array([data[sentence]['sentiment'] for sentence in sorted(data)])
punctuations = np.array([data[sentence]['punctuation'] for sentence in sorted(data)])
xs = np.array([ sentence.encode('utf-8') for sentence in sorted(data) ])
ys = np.array([ data[sentence]['label'] for sentence in sorted(data) ])

In [51]:
# concatenate features
features = [ x + ' ' + punct + ' ' + (' SUBREDDITx'.join(subreddit_list)).encode('utf-8') for subreddit_list, x, punct in zip(subreddits, xs, punctuations)]

#Uncomment this and comment the one above to reproduce byron's results
#features = [ x + ' ' + punct for x, punct in zip(xs, punctuations)]

print len(xs)

2748


# Load Progressive Comments for Progressivy Feature

In [52]:
cd /{base}/data/progressive/features/

/v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/data/progressive/features


In [53]:
import pickle
import scipy
import numpy as np

with open('text-sentiment-label.p', 'r') as f:
    data = pickle.load(f)

# Load comments, labels, and bow vectors
progressive_sentences = np.array([ comment for comment in data ])
progressive_sentiments = np.array([ data[comment]['sentiment'] for comment in data ])
progressive_labels = np.array([ data[comment]['label'] for comment in data ])

# Build Genuine Progressive Language Model

In [54]:
cd /{base}/lib/berkeleylm-1.1.5/examples

/v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/lib/berkeleylm-1.1.5/examples


In [57]:
from nltk.tokenize import word_tokenize

# Filter down to just genuine comments and tokenize them
genuine_sentences = [ x for x, y in zip(progressive_sentences, progressive_labels) if y == -1 ]
genuine_sentences = [ ' '.join(word_tokenize(genuine_sentence)) for genuine_sentence in genuine_sentences ]

# Write the genuine tokenized comments to disk so the Berkeley N-Gram Language model can be trained
with open('genuine-progressive.txt', 'w') as f:
    for genuine_sentence in genuine_sentences:
        f.write(genuine_sentence.encode('utf-8') + '\n')

# Make an arpa straight from text
!java -ea -mx1000m -server -cp ../src edu.berkeley.nlp.lm.io.MakeKneserNeyArpaFromText 5 genuine-progressive.arpa genuine-progressive.txt
print 'Finish building progressive ARPA from text!'

# Make a binary from the arpa
!java -ea -mx1000m -server -cp ../src edu.berkeley.nlp.lm.io.MakeLmBinaryFromArpa genuine-progressive.arpa genuine-progressive.binary

Reading text files [genuine-progressive.txt] and writing to file genuine-progressive.arpa {
	Reading in ngrams from raw text {
		On line 0
	} [0s]
	Writing Kneser-Ney probabilities {
		Counting counts for order 0 {
		} [0s]
		Counting counts for order 1 {
		} [0s]
		Counting counts for order 2 {
		} [0s]
		Counting counts for order 3 {
		} [0s]
		Counting counts for order 4 {
		} [0s]
		Writing ARPA {
			On order 1
			Writing line 1
			On order 2
			Writing line 1
			Writing line 10001
			Writing line 20001
			On order 3
			Writing line 1
			Writing line 10001
			Writing line 20001
			Writing line 30001
			On order 4
			Writing line 1
			Writing line 10001
			Writing line 20001
			Writing line 30001
			On order 5
			Writing line 1
			Writing line 10001
			Writing line 20001
			Writing line 30001
		} [0s]
	} [0s]
Finish building progressive ARPA from text!
Reading Lm File genuine-progressive.arpa . . .  {
	Counting values {
		Parsing ARPA language model file {
			Reading 1-grams {
				R

# Vectorize Features

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=50000, binary=True, stop_words="english")
#print subreddits
X = vectorizer.fit_transform(features)

In [83]:
type(X)

scipy.sparse.csr.csr_matrix

In [9]:
print len(xs)
print len(ys)
vectorizer.get_feature_names()

2748
2748


[u'000',
 u'02',
 u'03',
 u'04',
 u'040',
 u'05',
 u'050',
 u'07',
 u'08',
 u'09',
 u'10',
 u'100',
 u'100k',
 u'11',
 u'118',
 u'11e2',
 u'11was',
 u'12',
 u'120',
 u'1200',
 u'1297',
 u'13',
 u'1342',
 u'135',
 u'142',
 u'14pm',
 u'15',
 u'150',
 u'169',
 u'17',
 u'172',
 u'175',
 u'176565745',
 u'17th',
 u'18',
 u'19',
 u'190',
 u'1940',
 u'1944',
 u'1948',
 u'1952',
 u'1968',
 u'1982',
 u'1991',
 u'1995',
 u'1pfyb2',
 u'1rlvxk',
 u'1st',
 u'1trillion',
 u'20',
 u'200',
 u'2002',
 u'2007',
 u'2008',
 u'2009',
 u'200k',
 u'2010',
 u'2011',
 u'2012',
 u'2013',
 u'2014',
 u'2016',
 u'2020',
 u'204',
 u'21213547',
 u'22',
 u'222249311',
 u'228',
 u'229635361',
 u'23',
 u'246',
 u'25',
 u'250',
 u'2500',
 u'255',
 u'26',
 u'26fe',
 u'27',
 u'28',
 u'29',
 u'2k',
 u'2nd',
 u'30',
 u'300',
 u'3000',
 u'31',
 u'33',
 u'339',
 u'34',
 u'3403',
 u'350',
 u'359957',
 u'37',
 u'3x',
 u'40',
 u'401307114770931712',
 u'404',
 u'41',
 u'42',
 u'420',
 u'426',
 u'429',
 u'43554480',
 u'44173',
 u'4

# Compute Progressiviness Probabilities

In [60]:
from nltk.tokenize import word_tokenize

probs = [0]*len(xs)
print 'Must get to: {}'.format(len(xs))
for i, sentence in enumerate(xs):
    print i
    sentence = ' '.join(word_tokenize(sentence))
    out = !echo "{sentence}" | java -ea -mx1000m -server -cp ../src edu.berkeley.nlp.lm.io.ComputeLogProbabilityOfTextStream genuine-progressive.binary 2>&1 | tail -n 1
    print out
    prob = float(out[0].split()[5])
    probs[i] = prob
print 'Finish extracting conservativiness for training!'

Must get to: 2748
0
['Log probability of text is: -2.7554149627685547']
1
['Log probability of text is: -0.7582340240478516']
2
['Log probability of text is: -1.2535899877548218']
3
['Log probability of text is: -4.42034387588501']
4
['Log probability of text is: -1.5899529457092285']
5
['Log probability of text is: -2.9524409770965576']
6
['Log probability of text is: -0.9204170107841492']
7
['Log probability of text is: -2.50392484664917']
8
['Log probability of text is: -1.3349950313568115']
9
['Log probability of text is: -1.4814220666885376']
10
['Log probability of text is: -1.3349950313568115']
11
['Log probability of text is: -0.8501530289649963']
12
['Log probability of text is: -2.1371779441833496']
13
['Log probability of text is: -1.3349950313568115']
14
['Log probability of text is: -1.3349950313568115']
15
['Log probability of text is: -2.4244449138641357']
16
['Log probability of text is: -2.1371779441833496']
17
['Log probability of text is: -1.2054059505462646']
18
['L

# Tack on Progressiviness Feature

In [86]:
from scipy.sparse import coo_matrix, csr_matrix

# Convert progressiviness to column vector and smush onto the end of design matrix
progressivinesses = np.array([ [prob*sentiment] for prob, sentiment in zip(probs, sentiments) ])
progressivinesses = coo_matrix(progressivinesses)

X = csr_matrix(scipy.sparse.hstack([X, progressivinesses]))

# 5-Fold Test

In [91]:
from sklearn.cross_validation import KFold

kf = KFold(len(xs), n_folds=5, shuffle=True)

In [ ]:
from scipy.sparse import coo_matrix

from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier

recalls, precisions, f_measures = [], [], []
for train, test in kf:
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto", alpha=.01)
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(X[train], ys[train])

    # Make predictions
    predictions = clf.predict(X[test])
    
    for test, vector, prediction, actual in zip(xs[test], X[test], predictions, ys[test]):
        print '========================================================================'
        print
        print 'Comment: {}'.format(test)
        print 'Vectorized as: {}'.format(vector)
        print 'Prediction: {}'.format(prediction)
        print 'Was really: {}'.format(actual)
        print


Comment: 


oh and please don't correct my post of the improper use of capitalization because [that be racist!
Vectorized as:   (0, 5748)	2.9524409771
  (0, 5747)	2.9524409771
  (0, 5467)	1.0
  (0, 5031)	1.0
  (0, 4980)	1.0
  (0, 4954)	1.0
  (0, 4911)	1.0
  (0, 4801)	1.0
  (0, 4765)	1.0
  (0, 4761)	1.0
  (0, 4648)	1.0
  (0, 4647)	1.0
  (0, 4646)	1.0
  (0, 3807)	1.0
  (0, 3757)	1.0
  (0, 3565)	1.0
  (0, 3268)	1.0
  (0, 2425)	1.0
  (0, 1497)	1.0
  (0, 1143)	1.0
  (0, 776)	1.0
  (0, 420)	1.0
Prediction: 1
Was really: -1


Comment: 


oh sorry, he wouldn't so obviously that fuckhead in the crackhouse won't care.
Vectorized as:   (0, 5748)	0.920417010784
  (0, 5747)	0.920417010784
  (0, 5700)	1.0
  (0, 5667)	1.0
  (0, 5031)	1.0
  (0, 4761)	1.0
  (0, 4690)	1.0
  (0, 4446)	1.0
  (0, 3541)	1.0
  (0, 3268)	1.0
  (0, 3249)	1.0
  (0, 1963)	1.0
  (0, 1177)	1.0
  (0, 784)	1.0
Prediction: 1
Was really: -1


Comment: 

&gt; Both of those programs, however, are public, with no profit motive.
Vectoriz

# Most Indicative Negative Features

In [15]:
sgd_clf = clf.best_estimator_
feat_names = vectorizer.get_feature_names()
coefs_with_fns = sorted(zip(sgd_clf.coef_[0], feat_names))
coefs_with_fns[:20]

[(-0.1138284885109184, u'subredditxscience'),
 (-0.10237446581940357, u'subredditxpolitics'),
 (-0.081063693831936373, u'subredditxiama'),
 (-0.050814811651664855, u'subredditxbooks'),
 (-0.04670990178534154, u'atheism'),
 (-0.045518153759634833, u'subredditxliberal'),
 (-0.042431195331935942, u'subredditxprogressive'),
 (-0.041330483613748491, u'subredditxfood'),
 (-0.036778668237785077, u'subredditxentertainment'),
 (-0.036166242169019001, u'subredditxoffbeat'),
 (-0.034974494143312274, u'subredditxfantasyfootball'),
 (-0.034873526602244957, u'subredditxjusticeporn'),
 (-0.034693109192798187, u'subredditxscifi'),
 (-0.032499961784379472, u'subredditxdemocrats'),
 (-0.032144092582258685, u'subredditxmorbidreality'),
 (-0.031142693199546725, u'subredditxstlouis'),
 (-0.031101313059765396, u'subredditxfitness'),
 (-0.027219855948261901, u'bodyacceptance'),
 (-0.027219855948261901, u'subredditxbooksuggestions'),
 (-0.027219855948261901, u'subredditxdmb')]

# Most Indicative Features

In [18]:
coefs_with_fns[-1:-20:-1]

[(0.12176023370421923, u'subredditxadviceanimals'),
 (0.096197238552808331, u'politics'),
 (0.085188466165350321, u'puncxexclamation_point'),
 (0.080042431982124337, u'subredditxwtf'),
 (0.07421445309530024, u'puncxquestion_mark'),
 (0.058286409690610781, u'subredditxskyrim'),
 (0.058286409690610781, u'subredditxgaming4gamers'),
 (0.057791503218827915, u'subredditxatheism'),
 (0.053186721263943283, u'subredditxvideos'),
 (0.048224414901346248, u'subredditxaww'),
 (0.047037632492415195, u'subredditxfunny'),
 (0.042868169608032088, u'subredditxgaming'),
 (0.039081059215227275, u'just'),
 (0.039067817570497335, u'subredditxdoesanybodyelse'),
 (0.038885744955458747, u'subredditxgadgets'),
 (0.037887655983929201, u'subredditxhomeimprovement'),
 (0.034072407096076111, u'subredditxmapporn'),
 (0.033820815846205367, u'subredditxpics'),
 (0.033815850229430851, u'obamacare')]

# Baseline (Precision=0.19, Recall=0.50, F-Measure=0.24)

In [44]:
import sklearn

precisions, recalls, f_measures = [], [], []
for train, test in kf:
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto")
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(X[train], ys[train])

    # Make predictions
    predictions = clf.predict(X[test])
    
    # Record statistics
    precision, recall, f_measure, _ = sklearn.metrics.precision_recall_fscore_support(ys[test], predictions, average='binary')
    precisions.append(precision)
    recalls.append(recall)
    f_measures.append(f_measure)
    
print 'Precisions: {}'.format(precisions)
print 'Recalls: {}'.format(recalls)
print 'F-Measures: {}'.format(f_measures)
print
print 'Mean Precision: {}'.format(np.mean(precisions))
print 'Mean Recall: {}'.format(np.mean(recalls))
print 'Mean F-Measure: {}'.format(np.mean(f_measures))

Precisions: [0.091428571428571428, 0.058823529411764705, 0.083333333333333329, 0.090909090909090912, 0.069204152249134954]
Recalls: [0.47058823529411764, 0.88, 0.25925925925925924, 0.5, 0.7142857142857143]
F-Measures: [0.15311004784688997, 0.11027568922305764, 0.12612612612612614, 0.15384615384615385, 0.12618296529968454]

Mean Precision: 0.0787397354664
Mean Recall: 0.564826641768
Mean F-Measure: 0.133908196468
